- np.concatenate((a,b,c...))<br\>
- np.append(arr,values)<br\>
- df.columns.map(func)<br\>
- df.columns.append()<br\>
- df.append()

In [1]:
 #首先要引入各个包
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib
import os
%matplotlib inline

In [2]:
def select_col(fn,is_print=False):
    df = pd.read_excel(fn)
    columns = ['AST', 'BLK', 'BLKA', 'DREB', 'FG3A', 'FG3M', 'FGA', 'FGM', 
               'FTA','FTM','OREB', 'PF', 'PFD', 'PTS','PTS_2ND_CHANCE', 'PTS_FB', 
               'PTS_OFF_TOV', 'PTS_PAINT', 'REB','STL', 'TEAM_ABBREVIATION', 'TOV','GAME_ID','MATCHUP']
    return df[columns]

df = pd.DataFrame()
for fn in os.listdir('../数据收集'):
    if fn.endswith('.xlsx') and "games" not in fn:
        new_df = select_col('../数据收集/'+fn)
        df = pd.concat([df,new_df])
df.head(10)

,AST,BLK,BLKA,DREB,FG3A,FG3M,FGA,FGM,FTA,FTM,...,PTS_2ND_CHANCE,PTS_FB,PTS_OFF_TOV,PTS_PAINT,REB,STL,TEAM_ABBREVIATION,TOV,GAME_ID,MATCHUP
0,19,4,7,29,29,11,72,30,19,15,...,3,0,3,30,37,5,ATL,20,21601226,ATL @ IND
1,30,4,1,39,27,7,79,41,18,14,...,9,14,14,50,50,6,ATL,17,21601212,ATL vs. CHA
2,31,4,3,35,31,9,94,44,37,29,...,6,9,20,58,45,8,ATL,12,21601197,ATL vs. CLE
3,39,3,3,36,37,16,89,45,8,8,...,7,16,15,42,42,7,ATL,11,21601179,ATL @ CLE
4,26,4,4,39,23,11,89,44,34,24,...,25,17,14,44,52,7,ATL,17,21601177,ATL vs. BOS
5,19,4,6,36,24,7,80,30,23,15,...,6,15,28,34,44,9,ATL,20,21601148,ATL @ BKN
6,25,1,4,34,28,11,90,40,17,13,...,17,17,15,46,45,4,ATL,13,21601138,ATL @ CHI
7,19,3,3,32,18,9,80,34,26,22,...,21,0,10,38,47,7,ATL,17,21601111,ATL @ PHI
8,24,6,6,42,27,6,85,34,29,21,...,8,7,12,36,51,5,ATL,19,21601106,ATL vs. PHX
9,16,3,7,32,27,5,98,34,32,19,...,30,20,21,46,58,10,ATL,21,21601088,ATL vs. BKN


In [3]:
games = sorted(df['GAME_ID'].unique())
group_df = df.set_index('TEAM_ABBREVIATION').groupby('GAME_ID')
df.columns

Index(['AST', 'BLK', 'BLKA', 'DREB', 'FG3A', 'FG3M', 'FGA', 'FGM', 'FTA',
       'FTM', 'OREB', 'PF', 'PFD', 'PTS', 'PTS_2ND_CHANCE', 'PTS_FB',
       'PTS_OFF_TOV', 'PTS_PAINT', 'REB', 'STL', 'TEAM_ABBREVIATION', 'TOV',
       'GAME_ID', 'MATCHUP'],
      dtype='object')

In [4]:
example_group = group_df.get_group(games[0])
example_group.index

Index(['CLE', 'NYK'], dtype='object', name='TEAM_ABBREVIATION')

In [17]:
columns = example_group.loc[example_group.index[0]].index
columns = columns.append(columns.map(lambda x:"OPP_"+x))
columns

Index(['AST', 'BLK', 'BLKA', 'DREB', 'FG3A', 'FG3M', 'FGA', 'FGM', 'FTA',
       'FTM', 'OREB', 'PF', 'PFD', 'PTS', 'PTS_2ND_CHANCE', 'PTS_FB',
       'PTS_OFF_TOV', 'PTS_PAINT', 'REB', 'STL', 'TOV', 'GAME_ID', 'MATCHUP',
       'OPP_AST', 'OPP_BLK', 'OPP_BLKA', 'OPP_DREB', 'OPP_FG3A', 'OPP_FG3M',
       'OPP_FGA', 'OPP_FGM', 'OPP_FTA', 'OPP_FTM', 'OPP_OREB', 'OPP_PF',
       'OPP_PFD', 'OPP_PTS', 'OPP_PTS_2ND_CHANCE', 'OPP_PTS_FB',
       'OPP_PTS_OFF_TOV', 'OPP_PTS_PAINT', 'OPP_REB', 'OPP_STL', 'OPP_TOV',
       'OPP_GAME_ID', 'OPP_MATCHUP'],
      dtype='object')

In [18]:
feature_df = pd.DataFrame()
for game in games:
    game_df = group_df.get_group(game)
    one = game_df.loc[game_df.index[0]]
    two = game_df.loc[game_df.index[1]]
    feature_df = feature_df.append(pd.DataFrame(np.concatenate((one.values,two.values)),index=columns,columns = [game]).T)
    feature_df = feature_df.append(pd.DataFrame(np.concatenate((two.values,one.values)),index=columns,columns = [game]).T)
feature_df.head()

,AST,BLK,BLKA,DREB,FG3A,FG3M,FGA,FGM,FTA,FTM,...,OPP_PTS,OPP_PTS_2ND_CHANCE,OPP_PTS_FB,OPP_PTS_OFF_TOV,OPP_PTS_PAINT,OPP_REB,OPP_STL,OPP_TOV,OPP_GAME_ID,OPP_MATCHUP
21600001,31,5,6,40,35,13,94,45,19,14,...,88,16,9,14,28,42,6,18,21600001,NYK @ CLE
21600001,17,6,5,29,27,9,87,32,20,15,...,117,16,16,21,52,51,12,15,21600001,CLE vs. NYK
21600002,22,3,5,29,19,13,75,39,22,22,...,104,4,8,8,50,31,9,14,21600002,UTA @ POR
21600002,19,5,3,25,24,8,82,40,16,16,...,113,15,4,20,34,34,5,13,21600002,POR vs. UTA
21600003,24,6,3,27,33,7,85,40,18,13,...,129,26,24,17,50,55,13,14,21600003,SAS @ GSW


In [28]:
for col in ['OPP_MATCHUP','GAME_ID','OPP_GAME_ID']:
    if col in feature_df.columns:
        del feature_df[col]
feature_df.to_excel('features.xlsx')